Workflow
1. text preprocessing (tokenization, normalization). test spacy, stanza, pumorphy, llms 
2. extract potential wordd pairs (orthographic similarity)
3. craft the prompt, explain false friends and cognates 
4. interpret results 
5. post processing

In [1]:
ukr_text = 'texts/ukr.txt'
pol_text = 'texts/pl.txt'

with open(ukr_text, 'r') as f: 
    ukr = f.read()

with open(pol_text, 'r') as f:
    pol = f.read()

## Test NLP libraries

In [33]:
# Define a function to calculate the Jaro similarity 

from math import floor 

def jaro_distance(s1, s2):
     
    # If the s are equal
    if (s1 == s2):
        return 1.0
 
    # Length of two s
    len1 = len(s1)
    len2 = len(s2)
 
    # Maximum distance upto which matching
    # is allowed
    max_dist = floor(max(len1, len2) / 2) - 1
 
    # Count of matches
    match = 0
 
    # Hash for matches
    hash_s1 = [0] * len(s1)
    hash_s2 = [0] * len(s2)
 
    # Traverse through the first
    for i in range(len1):
 
        # Check if there is any matches
        for j in range(max(0, i - max_dist),
                       min(len2, i + max_dist + 1)):
             
            # If there is a match
            if (s1[i] == s2[j] and hash_s2[j] == 0):
                hash_s1[i] = 1
                hash_s2[j] = 1
                match += 1
                break
 
    # If there is no match
    if (match == 0):
        return 0.0
 
    # Number of transpositions
    t = 0
    point = 0
 
    # Count number of occurrences
    # where two characters match but
    # there is a third matched character
    # in between the indices
    for i in range(len1):
        if (hash_s1[i]):
 
            # Find the next matched character
            # in second
            while (hash_s2[point] == 0):
                point += 1
 
            if (s1[i] != s2[point]):
                t += 1
            point += 1
    t = t//2
 
    # Return the Jaro Similarity
    return (match/ len1 + match / len2 +
            (match - t) / match)/ 3.0

In [ ]:
# !pip install spacy stanza

In [ ]:
# !python3 -m spacy download pl_core_news_sm
# !python3 -m spacy download uk_core_news_sm
# !python3 -m spacy download uk_core_news_trf

In [1]:
import spacy 

nlp = spacy.load("pl_core_news_sm")

doc = nlp(u"Ukraina jest najlepszym krajem na świecie, moja Ukraina. Moja matka jest najlepszą kobietą na świecie.")

tokens_dict = {token.text: token.lemma_ for token in doc if token.is_alpha}

pl_lemmas = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]

# for token in doc:
#     print(token.text, token.lemma_, token.is_alpha, token.is_stop)

In [ ]:
pl_lemmas

In [4]:
import spacy 

nlp = spacy.load("uk_core_news_sm")

doc = nlp(u"Україна - то найкраща країна на всьому світі. Люблю її безмежно. Матка - то важливий жіночий орган.")

tokens_dict = {token.text: token.lemma_ for token in doc if token.is_alpha}

uk_lemmas = [token.lemma_ for token in doc if token.is_alpha]

# for token in doc:
#     print(token.text, token.lemma_, token.is_alpha, token.is_title)

In [5]:
uk_lemmas

['україна',
 'то',
 'найкращий',
 'країна',
 'на',
 'ввесь',
 'світ',
 'любити',
 'вона',
 'безмежно',
 'матка',
 'то',
 'важливий',
 'жіночий',
 'орган']

In [6]:
pl_lemmas

['Ukraina',
 'być',
 'dobry',
 'kraj',
 'na',
 'świat',
 'mój',
 'Ukraina',
 'mój',
 'matka',
 'być',
 'najlepszą',
 'kobieta',
 'na',
 'świat']

In [65]:
# !pip install Levenshtein polyglot icu tkinter

ERROR: Could not find a version that satisfies the requirement tkinter (from versions: none)
ERROR: No matching distribution found for tkinter


In [47]:
import Levenshtein
from polyglot.downloader import downloader 
from polyglot.text import Text 

# Ukrainian and Polish word lists
ukrainian_words = ['україна', 'то', 'найкращий', 'країна', 'на', 'ввесь', 'світ', 'любити', 'вона', 'безмежно', 'матка', 'то', 'важливий', 'жіночий', 'орган', "дискусія", "пироги"]
polish_words = ['Ukraina', 'być', 'dobry', 'kraj', 'na', 'świat', 'mój', 'Ukraina', 'mój', 'matka', 'być', 'najlepszą', 'kobieta', 'na', 'świat', "dyskusja", "pierogi"]

uk_trans = [str(Text(word.lower(), hint_language_code='uk').transliterate('en')[0]) for word in ukrainian_words]
pl_trans = [str(Text(word.lower(), hint_language_code='pl').transliterate('en')[0]) for word in polish_words]

# # Normalize the words to lowercase
# ukrainian_words = [word.lower() for word in ukrainian_words]
# polish_words = [word.lower() for word in polish_words]

# Set a threshold for similarity
# Example threshold is set to 3, adjust based on your needs

# Find pairs based on Levenshtein distance
similar_pairs = []
for u_word in uk_trans:
    for p_word in pl_trans:
        if jaro_distance(u_word, p_word) >= 0.8:
            similar_pairs.append((u_word, p_word))

# Print the similar pairs
print("Orthographically similar word pairs:")
for pair in similar_pairs:
    print(pair)


Orthographically similar word pairs:
('ukraina', 'ukraina')
('ukraina', 'ukraina')
('kraina', 'ukraina')
('kraina', 'ukraina')
('na', 'na')
('na', 'na')
('matka', 'matka')
('diskusia', 'dyskusja')
('pirogi', 'pierogi')


In [50]:
import Levenshtein
from polyglot.downloader import downloader 
from polyglot.text import Text 

# Ukrainian and Polish word lists
ukrainian_words = ['україна', 'то', 'найкращий', 'країна', 'на', 'ввесь', 'світ', 'любити', 'вона', 'безмежно', 'матка', 'то', 'важливий', 'жіночий', 'орган', "дискусія", "пироги"]
polish_words = ['Ukraina', 'być', 'dobry', 'kraj', 'na', 'świat', 'mój', 'Ukraina', 'mój', 'matka', 'być', 'najlepszą', 'kobieta', 'na', 'świat', "dyskusja", "pierogi"]

uk_trans = {str(Text(word.lower(), hint_language_code='uk').transliterate('en')[0]):word for word in ukrainian_words}
pl_trans = {str(Text(word.lower(), hint_language_code='pl').transliterate('en')[0]):word for word in polish_words}

threshold = 0.8

# Find pairs based on Levenshtein distance
similar_pairs = []
for u_word in uk_trans:
    for p_word in pl_trans:
        if jaro_distance(u_word, p_word) >= threshold:
            similar_pairs.append((uk_trans[u_word], pl_trans[p_word]))

# Print the similar pairs
print("Orthographically similar word pairs:")
for pair in similar_pairs:
    print(pair)

Orthographically similar word pairs:
('україна', 'Ukraina')
('країна', 'Ukraina')
('на', 'na')
('матка', 'matka')
('дискусія', 'dyskusja')
('пироги', 'pierogi')


In [51]:
similar_pairs

[('україна', 'Ukraina'),
 ('країна', 'Ukraina'),
 ('на', 'na'),
 ('матка', 'matka'),
 ('дискусія', 'dyskusja'),
 ('пироги', 'pierogi')]

In [52]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 KB 2.2 MB/s eta 0:00:00 MB/s eta 0:00:01:01
  Using cached anyio-4.3.0-py3-none-any.whl (85 kB)
  Using cached httpx-0.27.0-py3-none-any.whl (75 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
  Using cached distro-1.9.0-py3-none-any.whl (20 kB)
  Using cached httpcore-1.0.5-py3-none-any.whl (77 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)


In [60]:
similar_pairs = [('мир', 'mir'), ('рак', 'rak'), ('магазин', 'magazyn')]
similar_pairs

[('мир', 'mir'), ('рак', 'rak'), ('магазин', 'magazyn')]

In [63]:
import openai
import os 
from openai import OpenAI

client = OpenAI(
    api_key='sk-proj-L7S2GeXJ5B1hdpPfT1TVT3BlbkFJF8Iw3SjL6U2IgdvXjnyX'
)

# Assume you have a function that finds similar words between Ukrainian and Polish texts
potential_pairs = similar_pairs

def check_false_friends(u_word, p_word):
    response = client.chat.completions.create(
        messages = [
        {
        'role':'user',
        'content':f"Are Ukrainian word: '{u_word}' and Polish word: '{p_word}' false friends or cognates? Explain your answer."
        }
        ],
        model="gpt-4-turbo",  
    )
    return response
    return response.choices[0].text.strip()

In [64]:
results = {pair: check_false_friends(*pair) for pair in potential_pairs}
print(results)

{('мир', 'mir'): ChatCompletion(id='chatcmpl-9LY8vUmGk6Fm7NHK8LvuUkrizRSux', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The Ukrainian word "мир" and the Polish word "mir" can indeed be considered both false friends and cognates, though the specific meanings and usages in each language lead to different interpretations.\n\n1. **Cognates**: Both words are derived from the Old Church Slavonic word "миръ" which means "peace". Old Church Slavonic has influenced many Slavic languages, including Ukrainian and Polish. Therefore, if we look at the origin of the words, they stem from the same root and are thus cognates.\n\n2. **False Friends**: Despite having the same origin, the meaning of "мир" in contemporary Ukrainian and Polish can differ significantly, leading them to be false friends. In Ukrainian, "мир" primarily means "peace" (similar to its original Old Church Slavic meaning). However, it can also mean "world", in contexts simil

In [65]:
results

{('мир',
  'mir'): ChatCompletion(id='chatcmpl-9LY8vUmGk6Fm7NHK8LvuUkrizRSux', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The Ukrainian word "мир" and the Polish word "mir" can indeed be considered both false friends and cognates, though the specific meanings and usages in each language lead to different interpretations.\n\n1. **Cognates**: Both words are derived from the Old Church Slavonic word "миръ" which means "peace". Old Church Slavonic has influenced many Slavic languages, including Ukrainian and Polish. Therefore, if we look at the origin of the words, they stem from the same root and are thus cognates.\n\n2. **False Friends**: Despite having the same origin, the meaning of "мир" in contemporary Ukrainian and Polish can differ significantly, leading them to be false friends. In Ukrainian, "мир" primarily means "peace" (similar to its original Old Church Slavic meaning). However, it can also mean "world", in contexts sim

In [116]:
similar_pairs = [('Україна', 'Ukraina'), ('на', 'na'), ('матка', 'matka'),('пироги','pierogi'),('склеп','sklep')]

In [117]:
similar_pairs

[('Україна', 'Ukraina'),
 ('на', 'na'),
 ('матка', 'matka'),
 ('пироги', 'pierogi'),
 ('склеп', 'sklep')]

In [118]:
import openai
import os 
from openai import OpenAI

client = OpenAI(
    api_key='sk-proj-L7S2GeXJ5B1hdpPfT1TVT3BlbkFJF8Iw3SjL6U2IgdvXjnyX'
)

# Assume you have a function that finds similar words between Ukrainian and Polish texts
potential_pairs = similar_pairs

def check_false_friends(word_pairs):

    template = """Task: Determine if the following pairs of Ukrainian and Polish words are false friends (different meanings) or cognates (similar meanings).

    Example:
    {
    "word_pairs": [
        {
        "words": ["рак", "rak"],
        "status": "Cognates"
        },
        {
        "words": ["магазин", "magazyn"],
        "status": "False friends"
        }
    ]
    }

    Return the valid JSON as above for the following Ukrainian-Polish word pairs: """ 

    template += str(word_pairs)


    response = client.chat.completions.create(
        model="gpt-4-turbo-preview",
        response_format={"type":"json_object"},
        messages = [
            {
            'role':'user',
            'content':template
            }
        ]
    )
    return response
    return response.choices[0].text.strip()

In [119]:
res = check_false_friends(similar_pairs)

In [1]:
print(res.choices[0].message.content)

NameError: name 'res' is not defined

In [122]:
import json 

json_str = res.choices[0].message.content 

data = json.loads(json_str)

print(data)

{'word_pairs': [{'words': ['Україна', 'Ukraina'], 'status': 'Cognates'}, {'words': ['на', 'na'], 'status': 'Cognates'}, {'words': ['матка', 'matka'], 'status': 'False friends'}, {'words': ['пироги', 'pierogi'], 'status': 'Cognates'}, {'words': ['склеп', 'sklep'], 'status': 'False friends'}]}


In [72]:
template = """Task: Determine if the following pairs of Ukrainian and Polish words are false friends (different meanings) or cognates (similar meanings).

Examples:
1. Ukrainian: 'рак', Polish: 'rak' -> Cognates
2. Ukrainian: 'магазин', Polish: 'magazyn' -> False Friends

Classify the following pairs:""" 

for i, (ukr_word, pl_word) in enumerate(similar_pairs):
    template += f"{i}. Ukrainian: '{ukr_word}', Polish: '{pl_word}' -> \n"

In [73]:
print(template)

Task: Determine if the following pairs of Ukrainian and Polish words are false friends (different meanings) or cognates (similar meanings).

Examples:
1. Ukrainian: 'рак', Polish: 'rak' -> Cognates
2. Ukrainian: 'магазин', Polish: 'magazyn' -> False Friends

Classify the following pairs:0. Ukrainian: 'мир', Polish: 'mir' -> 
1. Ukrainian: 'рак', Polish: 'rak' -> 
2. Ukrainian: 'магазин', Polish: 'magazyn' -> 



In [38]:
print(jaro_distance('ukraina', 'kobieta'))

0.6190476190476191


In [11]:
!polyglot download transliteration2.uk
!polyglot download transliteration2.pl

[polyglot_data] Downloading package transliteration2.uk to
[polyglot_data]     /home/klychliiev/polyglot_data...
[polyglot_data]   Package transliteration2.uk is already up-to-date!
[polyglot_data] Downloading package transliteration2.pl to
[polyglot_data]     /home/klychliiev/polyglot_data...


In [15]:
from polyglot.downloader import downloader 
from polyglot.text import Text 

uk_blob = ukrainian_words[1]
pl_blob = polish_words[1]

uk_text = Text(uk_blob, hint_language_code='uk')
uk_translitareted = str(uk_text.transliterate('en')[0])


pl_text = Text(pl_blob, hint_language_code='pl')
pl_translitareted = str(pl_text.transliterate('en')[0])



In [17]:
pl_translitareted

'być'

In [26]:
import stanza 

uk_nlp = stanza.Pipeline(lang="uk", processors='tokenize,mwt,lemma')
pl_nlp = stanza.Pipeline(lang="pl", processors='tokenize,mwt,lemma')

2024-05-05 14:04:25 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-05-05 14:04:25 INFO: Downloaded file to /home/klychliiev/stanza_resources/resources.json
2024-05-05 14:04:25 INFO: Loading these models for language: uk (Ukrainian):
| Processor | Package     |
---------------------------
| tokenize  | iu          |
| mwt       | iu          |
| lemma     | iu_nocharlm |

2024-05-05 14:04:25 INFO: Using device: cpu
2024-05-05 14:04:25 INFO: Loading: tokenize
2024-05-05 14:04:25 INFO: Loading: mwt
2024-05-05 14:04:25 INFO: Loading: lemma
2024-05-05 14:04:25 INFO: Done loading processors!
2024-05-05 14:04:25 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-05-05 14:04:26 INFO: Downloaded file

In [38]:
doc = uk_nlp(u"Україна - то найкраща країна на всьому світі. Люблю її безмежно.")
print(*[f'{word.text}\t{word.lemma}\n' for sent in doc.sentences for word in sent.words])

In [30]:
doc = pl_nlp(u"Ukraina jest najlepszym krajem na świecie.")
print(*[f'{word.text}\t{word.lemma}\n' for sent in doc.sentences for word in sent.words])

Ukraina	ukraina
 jest	być
 najlepszym	dobry
 krajem	kraj
 na	na
 świecie	świat
 .	.



In [31]:
# !python3 -m spacy download pl_core_news_sm
!python3 -m spacy download uk_core_news_trf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.6 MB/s eta 0:00:00m eta 0:00:010:00:05
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.3/236.3 KB 1.1 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 4.6 MB/s eta 0:00:00m eta 0:00:010:01:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.6/731.6 KB 3.6 MB/s eta 0:00:00 MB/s eta 0:00:01:01
  Using cached regex-2024.4.28-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (774 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.26.1
    Uninstalling protobuf-5.26.1:
      Successfully uninstalled protobuf-5.26.1
✔ Download and installation successful
You can now load the package via spacy.load('uk_core_news_trf')


In [2]:
!pip install stanza

  Using cached stanza-1.8.2-py3-none-any.whl (990 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 189.0 kB/s eta 0:00:00m eta 0:00:010:00:41
  Using cached networkx-3.3-py3-none-any.whl (1.7 MB)
  Using cached protobuf-5.26.1-cp37-abi3-manylinux2014_x86_64.whl (302 kB)
  Using cached emoji-2.11.1-py2.py3-none-any.whl (433 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl (16 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 267.9 kB/s eta 0:00:00m eta 0:00:010:00:01
  Using cached fsspec-2024.3.1-py3-none-any.whl (171 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 319.7 kB/s eta 0:00:00m eta 0:00:010:00:12
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 212.5 kB/s eta 0:00:00m eta 0:00:010:00:28
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 KB 243.5 kB/s eta 0:00:00 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 807.2 kB/s eta 0:00:00m eta 0:00:010:01:36
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━